<a href="https://colab.research.google.com/github/jorell1/PyLZJD_Jupyter/blob/master/Implementing_LZJDh%2C_test_on_Shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing pyLZJD Clustering Shakespeare

In [25]:
from google.colab import drive
drive.mount("/content/drive")
!pip install git+git://github.com/EdwardRaff/pyLZJD#egg=pyLZJD

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
!ls drive/My\ Drive/Colab\ Notebooks/WS
dataDir = "drive/My Drive/Colab Notebooks/WS"
text_files = glob.glob(join(dataDir, "*.txt"))
print(text_files)


AllsWell.txt	    Henry8.txt		  Richard2.txt
AnthonyAndCleo.txt  JuliusCaesar.txt	  Richard3.txt
AsYouLikeIt.txt     KingJohn.txt	  RomeoJuliet.txt
ComedyOfErrors.txt  KingLear.txt	  TamingShrew.txt
Coriolanus.txt	    LoveLabors.txt	  Tempest.txt
Cymbeline.txt	    Macbeth.txt		  Timon.txt
Hamlet.txt	    Measure4Measure.txt   TitusAndronicus.txt
Henry4Part1.txt     MerchantOfVenice.txt  TroiluisCressida.txt
Henry4Part2.txt     MerryWives.txt	  TwelfthNight.txt
Henry5.txt	    MidsummerNights.txt   TwoGentlemenVerona.txt
Henry6Part1.txt     MuchAdo.txt		  WinterTale.txt
Henry6Part2.txt     Othello.txt
Henry6Part3.txt     Pericles.txt
['drive/My Drive/Colab Notebooks/WS/AsYouLikeIt.txt', 'drive/My Drive/Colab Notebooks/WS/ComedyOfErrors.txt', 'drive/My Drive/Colab Notebooks/WS/Coriolanus.txt', 'drive/My Drive/Colab Notebooks/WS/Cymbeline.txt', 'drive/My Drive/Colab Notebooks/WS/Henry4Part1.txt', 'drive/My Drive/Colab Notebooks/WS/Henry4Part2.txt', 'drive/My Drive/Colab Notebooks/WS/Henry6Pa

In [27]:
#wd = dropBoxDir+"working/vx/"

dataDir = "drive/My Drive/Colab Notebooks/WS"

import os, glob, time
from os.path import isfile, join
import numpy as np


from pyLZJD import sim, digest

text_files = glob.glob(join(dataDir, "*.txt"))
#text_files = [f for f in os.listdir(dataDir) if f.endswith('.txt')]
#print(text_files)
n = len(text_files)
result = digest(text_files)
print(text_files)


['drive/My Drive/Colab Notebooks/WS/AsYouLikeIt.txt', 'drive/My Drive/Colab Notebooks/WS/ComedyOfErrors.txt', 'drive/My Drive/Colab Notebooks/WS/Coriolanus.txt', 'drive/My Drive/Colab Notebooks/WS/Cymbeline.txt', 'drive/My Drive/Colab Notebooks/WS/Henry4Part1.txt', 'drive/My Drive/Colab Notebooks/WS/Henry4Part2.txt', 'drive/My Drive/Colab Notebooks/WS/Henry6Part1.txt', 'drive/My Drive/Colab Notebooks/WS/Henry6Part2.txt', 'drive/My Drive/Colab Notebooks/WS/Henry6Part3.txt', 'drive/My Drive/Colab Notebooks/WS/Henry8.txt', 'drive/My Drive/Colab Notebooks/WS/JuliusCaesar.txt', 'drive/My Drive/Colab Notebooks/WS/KingJohn.txt', 'drive/My Drive/Colab Notebooks/WS/KingLear.txt', 'drive/My Drive/Colab Notebooks/WS/LoveLabors.txt', 'drive/My Drive/Colab Notebooks/WS/Macbeth.txt', 'drive/My Drive/Colab Notebooks/WS/Measure4Measure.txt', 'drive/My Drive/Colab Notebooks/WS/MerchantOfVenice.txt', 'drive/My Drive/Colab Notebooks/WS/MerryWives.txt', 'drive/My Drive/Colab Notebooks/WS/MidsummerNights.t

Now compute the lzsets.  This needs to be done only once.  The lzset can be thought of as a set of strings such that all the strings occur in the lzset can be concatenedated, in some order and with the given amount of repetiton, to recreate the original document.  Also, each string in the lzset does occur in the document at least once, but if the frequency is zero, the string DOES occur at least once.  It was just not needed for the sake of LZ compression.  

In [28]:
#build a similarity matrix for use in hierarchical clustering
# sim2 will be upper triangular only!
nchoose2 = n*(n-1)//2
print("Number of files is %d" % n)
print("Number of pairs (n choose 2) is %d" % nchoose2)

import os, glob
from os.path import isfile, join

from pyLZJD import sim, digest

text_files = glob.glob(join(dataDir, "*.txt"))
#text_files = [f for f in os.listdir(dataDir) if f.endswith('.txt')]
#print(text_files)
n = len(text_files)
result = digest(text_files)

sim2 = np.zeros(nchoose2)
k = 0
shortestD = 100
f1 = f2 = ""
ishort = jshort = -1

for i in range(n-1):
    print("computing similarity matrix row %d/%d (%s)" % (i+1, n, text_files[i]))
    for j in range(i+1,n):
        theDistance = sim(result[i],result[j])
        sim2[k] = theDistance
        if  theDistance < shortestD:
            shortestD = theDistance
            f1 = text_files[i]
            ishort = i
            f2 = text_files[j]
            jshort = j
            print("Closest files so far, with distance of %f (%d, %d): %s and %s" % 
                  (theDistance, ishort, jshort, f1, f2))
        k += 1

Number of files is 37
Number of pairs (n choose 2) is 666
computing similarity matrix row 1/37 (drive/My Drive/Colab Notebooks/WS/AsYouLikeIt.txt)
Closest files so far, with distance of 0.179724 (0, 1): drive/My Drive/Colab Notebooks/WS/AsYouLikeIt.txt and drive/My Drive/Colab Notebooks/WS/ComedyOfErrors.txt
Closest files so far, with distance of 0.178366 (0, 3): drive/My Drive/Colab Notebooks/WS/AsYouLikeIt.txt and drive/My Drive/Colab Notebooks/WS/Cymbeline.txt
Closest files so far, with distance of 0.175660 (0, 4): drive/My Drive/Colab Notebooks/WS/AsYouLikeIt.txt and drive/My Drive/Colab Notebooks/WS/Henry4Part1.txt
Closest files so far, with distance of 0.159683 (0, 14): drive/My Drive/Colab Notebooks/WS/AsYouLikeIt.txt and drive/My Drive/Colab Notebooks/WS/Macbeth.txt
computing similarity matrix row 2/37 (drive/My Drive/Colab Notebooks/WS/ComedyOfErrors.txt)
Closest files so far, with distance of 0.124657 (1, 2): drive/My Drive/Colab Notebooks/WS/ComedyOfErrors.txt and drive/My D

In [29]:
print("According to LZJD, the two closest plays are (%d, %d): %s and %s, with a distance of %f" 
      % (ishort, jshort, f1, f2, shortestD))

According to LZJD, the two closest plays are (33, 34): drive/My Drive/Colab Notebooks/WS/Hamlet.txt and drive/My Drive/Colab Notebooks/WS/TwoGentlemenVerona.txt, with a distance of 0.117904


Granted that Henry 6 parts 2 and 3 are the most similar plays.  What strings do that have in common?  

In [30]:
text_files[11]

'drive/My Drive/Colab Notebooks/WS/KingJohn.txt'

In [31]:
text_files[12]

'drive/My Drive/Colab Notebooks/WS/KingLear.txt'